# Brave Video Search + YouTube Metadata (Single API Call)
This notebook fetches the first 10 YouTube videos from Brave's Video Search API and then retrieves metadata for all of them in a single YouTube Data API request.

In [ ]:
!pip install requests google-api-python-client

In [ ]:
import requests
from googleapiclient.discovery import build
import re

# API keys (replace with your own)
BRAVE_API_KEY = os.getenv("SECRET__BRAVE_SEARCH__API_KEY")
YOUTUBE_API_KEY = os.getenv("SECRET__YOUTUBE__API_KEY")

# Search query
query = "gpt5"

# Step 1: Brave Video Search API request (limit to 10 results)
brave_url = "https://api.search.brave.com/res/v1/videos/search"
headers = {"X-Subscription-Token": BRAVE_API_KEY, "Accept": "application/json"}
params = {
    "q": query,
    "count": 10,
    "country": "us",
    "search_lang": "en",
    "spellcheck": 1
}

brave_response = requests.get(brave_url, headers=headers, params=params)
brave_data = brave_response.json()

# Step 2: Extract YouTube video IDs
video_ids = []
## NOTE: get the first 15 videos and filter out for the first 10 youtube videos?
for item in brave_data.get("results", []):
    url = item.get("url", "")
    if "youtube.com/watch" in url:
        match = re.search(r"v=([a-zA-Z0-9_-]{11})", url)
        if match:
            video_ids.append(match.group(1))

print(f"Found {len(video_ids)} YouTube videos: {video_ids}")

# Step 3: Make a single YouTube API call for metadata
if video_ids:
    youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)
    request = youtube.videos().list(
        part="snippet,statistics",
        id=",".join(video_ids)
    )
    yt_response = request.execute()

    for item in yt_response["items"]:
        video_id = item["id"]
        title = item["snippet"]["title"]
        description = item["snippet"]["description"]
        channel_name = item["snippet"]["channelTitle"]  # Add channel name
        youtube_url = f"https://www.youtube.com/watch?v={video_id}"  # Create YouTube URL
        views = item["statistics"].get("viewCount", "N/A")
        likes = item["statistics"].get("likeCount", "N/A")
        comments = item["statistics"].get("commentCount", "N/A")
        
        print(f"Title: {title}")
        print(f"Channel: {channel_name}")  # Add this line
        print(f"URL: {youtube_url}")  # Add this line
        print(f"Description: {description[:200]}...")
        print(f"Views: {views} | Likes: {likes} | Comments: {comments}")
        print("-" * 50)
else:
    print("No YouTube videos found in Brave results.")

Found 10 YouTube videos: ['LWWbwgKfdNE', 'A30mVgbG-OQ', 'xx_5ADX21-4', '8tx2viHpgA8', '-gXmWYQtv5o', 'SJVCfPGsy30', '7w38FqMYA1E', 'WLdBimUS1IE', 'mczty3A0txQ', 'BUDmHYI6e3g']
Title: GPT-5 Full Breakdown! (Everything You Need to Know)
Channel: Matthew Berman
URL: https://www.youtube.com/watch?v=LWWbwgKfdNE
Description: Check out Box AI here: https://bit.ly/4m5EDLH

Download (GPT-5 UPDATED) Humanities Last Prompt Engineering Guide (free) 👇🏼
http://bit.ly/4m76knm

Join My Newsletter for Regular AI Updates 👇🏼
https://f...
Views: 29923 | Likes: 1013 | Comments: 227
--------------------------------------------------
Title: OpenAI dropped GPT-5, is AGI here?
Channel: IBM Technology
URL: https://www.youtube.com/watch?v=A30mVgbG-OQ
Description: Visit Mixture of Experts podcast page to get more AI content → https://ibm.biz/BdezK6

Is GPT-5 better at code than Claude Opus 4.1? In this bonus episode of Mixture of Experts, guest host Bryan Casey...
Views: 5672 | Likes: 181 | Comments: 11
---------